# 교차검증(Cross Validation,CV)

- 모델에 적절한 HyperParameter를 찾기 위해
- 과적합(Overfitting)예방 : 모델이 train 데이터에만 과도하게 최적화되어, 
  실제예측을 다른 데이터로 수행할 경우 예측성능이 과도하게 떨어지는 것

## KFold 교차검증 :  K개의 데이터 폴드 세트 생성 후 K번 만큼 학습과 검증평가를 반복 수행

1) KFold클래스 : 폴드 세트로 분리하는 객체 생성
    - kfold = KFold(n_splits=5)
    
2) split()메소드 : 폴드 데이터 세트로 분리
    - kfold.split(features)
    - 각 폴드마다 학습, 검증용 데이터로 분할할 수 있는 인덱스 반환
    - 학습,검증용 데이터 추출은 인덱스를 기반으로 개발코드에서 직접 수행
    
###  교차검증 과정

1) 폴드 세트 설정
2) for 반복문으로 학습 및 데이터 인덱스 추출
3) 반복적으로 학습과 예측 수행, 예측성능 평가

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target

# DecisionTreeClassfier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=156)
# 5개의 폴드 세트로 분리
kfold = KFold(n_splits=5)
# 폴드 세트별 정확도 담을 리스트 생성
cv_accuracy = []

print('붓꽃 데이터 세트 크기 :', features.shape)


붓꽃 데이터 세트 크기 : (150, 4)


KFold 객체의 split()을 호출해 교차검증 수행시마다 학습,검증을 반복해 예측 정확도 측정
- split이 실제 반환하는 값 확인을 위해 검증데이터 세트의 인덱스 추출

In [2]:
n_iter=0

# KFold 객체의 split()을 호출하면 폴드별 학습, 검증용 테스트의 로우인덱스 array로 반환

for train_index, test_index in kfold.split(features):
    # kfold.split()으로 반환된 인덱스를 이용해 학습, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    # 학습 
    dt_clf.fit(X_train,y_train)
    #예측
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    #반복 시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n # {0} 교차검증 정확도:{1}, 학습 데이터 크기 :{2}, 검증 데이터 크기:{3}'.format(n_iter,accuracy,train_size,test_size))
    print('# {0} 검증 세트 인덱스 :{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
print('\n ## 평균 검증 정확도:',np.mean(cv_accuracy))


 # 1 교차검증 정확도:1.0, 학습 데이터 크기 :120, 검증 데이터 크기:30
# 1 검증 세트 인덱스 :[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

 # 2 교차검증 정확도:0.9667, 학습 데이터 크기 :120, 검증 데이터 크기:30
# 2 검증 세트 인덱스 :[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

 # 3 교차검증 정확도:0.8667, 학습 데이터 크기 :120, 검증 데이터 크기:30
# 3 검증 세트 인덱스 :[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

 # 4 교차검증 정확도:0.9333, 학습 데이터 크기 :120, 검증 데이터 크기:30
# 4 검증 세트 인덱스 :[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

 # 5 교차검증 정확도:0.7333, 학습 데이터 크기 :120, 검증 데이터 크기:30
# 5 검증 세트 인덱스 :[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

 ## 평균 검증 정확도: 0.9


## Stratified K 폴드 

- 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K폴드 방식
    
    - 불균형한 레이블 집합 : 특정 레이블 값이 특이하게 많거나 적어 값이 한쪽으로 치우치는 것

1 먼저 kfold를 활용해 레이블 분포 확인

In [3]:
import pandas as pd
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [4]:
kfold = KFold(n_splits=3)
n_iter = 0

for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증 : ',n_iter)
    print('학습 레이블 분포 :\n',label_train.value_counts())
    print('검증 레이블 분포 :',label_test.value_counts())

## 교차검증 :  1
학습 레이블 분포 :
 1    50
2    50
Name: label, dtype: int64
검증 레이블 분포 : 0    50
Name: label, dtype: int64
## 교차검증 :  2
학습 레이블 분포 :
 0    50
2    50
Name: label, dtype: int64
검증 레이블 분포 : 1    50
Name: label, dtype: int64
## 교차검증 :  3
학습 레이블 분포 :
 0    50
1    50
Name: label, dtype: int64
검증 레이블 분포 : 2    50
Name: label, dtype: int64


In [5]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df,iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증 :{0}'.format(n_iter))
    print('학습 데이터 분포 :',label_train.value_counts())
    print('검증데이터 분포 :',label_test.value_counts())


## 교차검증 :1
학습 데이터 분포 : 2    34
0    33
1    33
Name: label, dtype: int64
검증데이터 분포 : 0    17
1    17
2    16
Name: label, dtype: int64
## 교차검증 :2
학습 데이터 분포 : 1    34
0    33
2    33
Name: label, dtype: int64
검증데이터 분포 : 0    17
2    17
1    16
Name: label, dtype: int64
## 교차검증 :3
학습 데이터 분포 : 0    34
1    33
2    33
Name: label, dtype: int64
검증데이터 분포 : 1    17
2    17
0    16
Name: label, dtype: int64


kfold는 학습레이블 이외 검증레이블은 전혀 학습을 하지 못함.
- StratifiedKFold는 이렇게 분할된 레이블 데이터세트가 전체 레이블 값의 
  분포도를 반영하지 못하는 문제를 해결

In [6]:

dt_clf = DecisionTreeClassifier(random_state=156)

skf = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy=[]

#Stratifiedkfold 의 split() 호출시 반드시 레이블 데이터 세트도 추가
for train_index, test_index in skf.split(features,label):
    
    # 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index],features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    # 반복 시 마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
 
    print('\n ## {0}교차검증 정확도:{1}, 학습용데이터 크기:{2}, 검증 데이터 크기:{3}'.format(n_iter,accuracy,train_size,test_size))

    # 교차 
    cv_accuracy.append(accuracy)
print('\n##교차검증별 정확도 :',np.round(cv_accuracy,4))
print('##평균 정확도 : ',np.mean(cv_accuracy))
    


 ## 1교차검증 정확도:0.98, 학습용데이터 크기:100, 검증 데이터 크기:50

 ## 2교차검증 정확도:0.94, 학습용데이터 크기:100, 검증 데이터 크기:50

 ## 3교차검증 정확도:0.98, 학습용데이터 크기:100, 검증 데이터 크기:50

##교차검증별 정확도 : [0.98 0.94 0.98]
##평균 정확도 :  0.9666666666666667


## cross_val_score() 함수로 교차검증을 간편하게

- 폴드 세트 설정, for 반복문, 폴드 세트별 예측 성능 평균으로 최종 성능 평가까지
  한번에 수행
  
- 내부 Estimator를 학습:fit, 예측:predict, 평가:evaluation 시켜줌

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

feature = iris.data
target = iris.target

# 성능 지표는 정확도accuracy, 교차검증 세트는 3개

scores = cross_val_score(dt_clf,feature,target,scoring='accuracy', cv=3)
print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도: ',np.round(np.mean(scores),4))


교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도:  0.9667


## GridSearchCV 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에

- Grid는 격자라는 의미 : 촘촘하게 파라미터를 입력하면서 테스트 하는 방식
- 파라미터의 집합을 만들고 순차적으로 적용해 최적화 수행
    - 순차적으로 최고성능을 가지는 파라미터를 찾을 때 활용
    
         -GridSearchCV 클래스 생성자의 주요 파라미터
          - Estimator : classfier, regressor, pipeline
          - param_grid : key + 리스트 값을 가지는 딕셔너리가 주어짐
          - scoring : 예측 성능 평가 방법 지정
          - cv : 교차 검증을 위해 분할할 학습, 테스트 세트 개수지정
          - refit : 디폴트가 True이며 True로 생성시 가장 최적의 하이퍼 파라미터 찾은 뒤 
                    입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습
                    
                    
 - GridSearchCV  fit(train데이터) 학습
 - cv에 기술된 폴딩으로 분할
 - param_grid에 기술된 하이퍼 파라미터를 순차적으로 학습, 수행
 - 결과를 cv_results 에 기록

In [22]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

#데이터 로딩 후 학습데이터, 테스트데이터 분리

iris = load_iris()
X_train,X_test,y_train,y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=121)

dt_clf = DecisionTreeClassifier()

##파라미터를 딕셔너리 형태로 설정

parameters = {'max_depth': [1, 2, 3], 'min_samples_split':[2, 3]}

In [25]:
## 하이퍼 파라미터를 3개의 train, test set fold 로 나누어 테스트 수행설정
## refit=True가 default, True 면 가장 좋은 파라미터 설정으로 재학습

grid_tree = GridSearchCV(dt_clf, param_grid=parameters, cv=3, refit=True, return_train_score=True)

#붓꽃 학습 데이터로 param_grid 하이퍼 파라미터 순차적으로 학습/평가
grid_tree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 DataFrame 으로 변환
scores_df = pd.DataFrame(grid_tree.cv_results_)
scores_df


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.000665,4.701341e-04,0.000333,0.000471,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
1,0.000997,3.371748e-07,0.000000,0.000000,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
2,0.000665,4.700779e-04,0.000000,0.000000,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
3,0.000664,4.702758e-04,0.000333,0.000471,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
4,0.001006,1.362722e-05,0.000000,0.000000,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02
5,0.000998,9.199649e-07,0.000000,0.000000,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02


In [28]:
print('파라미터 확인:',grid_tree.cv_results_)
print('GridSearchCV 최적의 파라미터:',grid_tree.best_params_)
print('GridSearchCV 최고 정확도:',grid_tree.best_score_)

파라미터 확인: {'mean_fit_time': array([0.00066487, 0.0009973 , 0.00066479, 0.00066439, 0.00100581,
       0.00099818]), 'std_fit_time': array([4.70134086e-04, 3.37174788e-07, 4.70077860e-04, 4.70275811e-04,
       1.36272203e-05, 9.19964862e-07]), 'mean_score_time': array([0.00033299, 0.        , 0.        , 0.00033275, 0.        ,
       0.        ]), 'std_score_time': array([0.00047092, 0.        , 0.        , 0.00047058, 0.        ,
       0.        ]), 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 1, 'min_samples_split': 2}, {'max_depth': 1, 'min_samples_split': 3}, {'max_depth': 2, 'min_samples_split': 2}, {'max_depth': 2, 'min_samples_split': 3}, {'max_depth': 3, 'm

refit=True : GridSearchCV가 최적 성능을 나타내는 하이퍼 파라미터로 Estimator를 학습해 best_estimator로 저장

In [29]:
# refit()으로 이미 학습된 estimator 반환(별로도 fit()학습 필요없음)
estimator = grid_tree.best_estimator_

pred = estimator.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('테스트 데이터 세트 정확도 : ', np.round(accuracy,4))

테스트 데이터 세트 정확도 :  0.9667
